In [1]:
import os
import gzip
import shutil
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

from matplotlib.ticker import MultipleLocator, AutoMinorLocator
from Bio.PDB import (
    PDBList,
    PDBIO,
    NeighborSearch,
    calc_angle,
    calc_dihedral,
    PPBuilder,
    is_aa,
)
import category_encoders as ce
from Bio.PDB.PDBParser import PDBParser
from Bio.SeqUtils import IUPACData, seq1
from Bio.PDB.PDBIO import Select
from Bio.SeqIO.PdbIO import PdbSeqresIterator

# Data Extraction

We'll use this notebook to create the datasets we are going to use.

All the datasets created here will be saved in `data` directory

In [9]:
path_to_data = Path("../data")  # Access to data folder

## All features

We start by extracting all the features from the table given from the professor

In [9]:
# Combine all PDBs into a single dataframe
ring_path = Path("../features_ring")

dfs = []
for filename in os.listdir(ring_path):
    dfs.append(pd.read_csv(ring_path / filename, sep="\t"))
df = pd.concat(dfs)

# Here we are removing the id that don't make sense
df = df[~df["pdb_id"].isin(set([id for id in df["pdb_id"] if type(id) != str]))]

df.sort_values(by=["pdb_id", "s_ch"], inplace=True)
df.reset_index(drop=True, inplace=True)

df.insert(df.columns.get_loc("Interaction"), "dist", np.nan)

print(df.shape)
df.head()

(2468259, 35)


,pdb_id,s_ch,s_resi,s_ins,s_resn,s_ss8,s_rsa,s_up,s_down,s_phi,...,t_phi,t_psi,t_ss3,t_a1,t_a2,t_a3,t_a4,t_a5,dist,Interaction
0,1aba,A,50,,E,H,0.304,7.0,10.0,-1.107,...,-1.102,-0.657,H,1.831,-0.561,0.533,-0.277,1.648,NaN,HBOND
1,1aba,A,67,,Q,E,0.086,14.0,24.0,-2.214,...,-1.350,2.242,H,0.336,-0.417,-1.673,-1.474,-0.078,NaN,HBOND
2,1aba,A,17,,C,H,0.000,17.0,17.0,-1.093,...,-1.043,-0.817,H,1.831,-0.561,0.533,-0.277,1.648,NaN,HBOND
3,1aba,A,45,,D,H,0.742,1.0,13.0,-1.182,...,-1.034,-0.718,H,-0.591,-1.302,-0.733,1.570,-0.146,NaN,HBOND
4,1aba,A,39,,E,B,0.634,0.0,15.0,-2.661,...,-2.073,2.008,H,-1.337,-0.279,-0.544,1.242,-1.262,NaN,HBOND


### Contact Map

In [4]:
def contact_map(pdb_id: str, chain_id: str, path: Path):
    # Create the directory if it doesn't exist
    if not os.path.exists(path):
        os.makedirs(path)

    # Retrieve the PDB file and save it in the specified directory
    PDBList().retrieve_pdb_file(pdb_id, file_format="pdb", pdir=path)

    try:
        structure = PDBParser(QUIET=True).get_structure(
            pdb_id, path / f"pdb{pdb_id}.ent"
        )
    except:
        return None

    selected_residues = [
        residue for residue in structure[0][chain_id] if residue.id[0] == " "
    ]

    distances = []
    for residue1 in selected_residues:
        row = []
        for residue2 in selected_residues:
            # Check sequence separation
            # if abs(residue1.id[1] - residue2.id[1]) >= seq_sep:
            try:
                row.append(residue1["CA"] - residue2["CA"])
            except:
                row.append(None)  # For residues not respecting sequence separation
        distances.append(row)
    return np.array(distances, dtype=float)

In [5]:
path_to_contact_maps = Path("../data/contact_maps")

prev_id = None
prev_ch = None

for i, (pdb_id, chain_id) in enumerate(zip(df["pdb_id"], df["s_ch"])):
    # Creating the map only when changing id or chain
    if prev_id != pdb_id or prev_ch != chain_id:
        # Creating the distance map
        dist_map = contact_map(
            pdb_id=pdb_id, chain_id=chain_id, path=path_to_contact_maps
        )
        # Updating the previous pdb_id and chain
        prev_id = pdb_id
        prev_ch = chain_id

        try:
            print(
                f"pdb_id: {pdb_id}; chain_id: {chain_id}\ndist_map.shape: {dist_map.shape}\n"
            )
        except:
            continue

    # Adding the distance in the dataframe
    try:
        df.loc[i, "dist"] = dist_map[df["s_resi"][i] - 1, df["t_resi"][i] - 1]
    except:
        continue


df.to_csv(path_to_data / "df.csv", index=False)

df.head()

Structure exists: '../data/contact_maps/pdb1aba.ent' 
pdb_id: 1aba; chain_id: A
dist_map.shape: (87, 87)

Structure exists: '../data/contact_maps/pdb1agi.ent' 
pdb_id: 1agi; chain_id: A
dist_map.shape: (125, 125)

Structure exists: '../data/contact_maps/pdb1b0y.ent' 
pdb_id: 1b0y; chain_id: A
dist_map.shape: (85, 85)

Structure exists: '../data/contact_maps/pdb1b67.ent' 
pdb_id: 1b67; chain_id: A
dist_map.shape: (68, 68)

Structure exists: '../data/contact_maps/pdb1b67.ent' 
pdb_id: 1b67; chain_id: B
dist_map.shape: (65, 65)

Structure exists: '../data/contact_maps/pdb1bte.ent' 
pdb_id: 1bte; chain_id: A
dist_map.shape: (92, 92)

Structure exists: '../data/contact_maps/pdb1bte.ent' 
pdb_id: 1bte; chain_id: B
dist_map.shape: (94, 94)

Structure exists: '../data/contact_maps/pdb1bw9.ent' 
pdb_id: 1bw9; chain_id: A
dist_map.shape: (350, 350)

Structure exists: '../data/contact_maps/pdb1bw9.ent' 
pdb_id: 1bw9; chain_id: B
dist_map.shape: (347, 347)

Structure exists: '../data/contact_maps/

,pdb_id,s_ch,s_resi,s_ins,s_resn,s_ss8,s_rsa,s_up,s_down,s_phi,...,t_phi,t_psi,t_ss3,t_a1,t_a2,t_a3,t_a4,t_a5,dist,Interaction
0,1aba,A,50,,E,H,0.304,7.0,10.0,-1.107,...,-1.102,-0.657,H,1.831,-0.561,0.533,-0.277,1.648,6.292980,HBOND
1,1aba,A,67,,Q,E,0.086,14.0,24.0,-2.214,...,-1.350,2.242,H,0.336,-0.417,-1.673,-1.474,-0.078,7.907008,HBOND
2,1aba,A,17,,C,H,0.000,17.0,17.0,-1.093,...,-1.043,-0.817,H,1.831,-0.561,0.533,-0.277,1.648,6.614870,HBOND
3,1aba,A,45,,D,H,0.742,1.0,13.0,-1.182,...,-1.034,-0.718,H,-0.591,-1.302,-0.733,1.570,-0.146,6.087462,HBOND
4,1aba,A,39,,E,B,0.634,0.0,15.0,-2.661,...,-2.073,2.008,H,-1.337,-0.279,-0.544,1.242,-1.262,5.471278,HBOND


## Data Augmentation

In [ ]:
# Function to modify some entries of a row
def mod_row(row, factor=10, idxs=[], extract=-1, verbose=False):
    # idxs by default are all
    if len(idxs) < 1:
        idxs = np.arange(0, row.shape[0])  # which indexes/columns can be modified?
    if extract < 0:
        extract = np.random.choice(
            len(idxs) + 1, 1
        )  # how many indexes do you want to mod? NB choice(5,1) is btw 0 and 4 included so must do 5+1 to select a 5 (all items in row)

    col_mutated = np.sort(
        np.random.choice(idxs, size=extract, replace=False)
    )  # select randomly which columns to modify

    mrow = row.copy()
    mrow[col_mutated] += np.random.uniform(
        -0.01, 0.01
    )  # we want to modify not more than 0.01 of the inital value

    if verbose:
        print(f"extract = {extract}\n, col_mutated = {col_mutated}\n, row = {row}")

    return mrow

In [ ]:
inter_count = df.groupby(["Interaction"]).size()
rare_inter = inter_count[inter_count < 100000]
oversamples = {
    "IONIC": 80000,
    "PICATION": 60000,
    "PIHBOND": 40000,
    "PIPISTACK": 80000,
    "SSBOND": 40000,
}
# Lis of columns to modify in data augumentation phase
col_to_mod = [
    "s_rsa",
    "s_up",
    "s_down",
    "s_phi",
    "s_psi",
    "t_rsa",
    "t_up",
    "t_down",
    "t_phi",
    "t_psi",
    "dist",
]
col_to_mod_idx = [
    index for index, value in enumerate(df.columns) if value in col_to_mod
]

data_aug = pd.DataFrame([])
DAUG_factor = 1.01  # conservative tweak

In [ ]:
# Data Augumentation
for inter in rare_inter.index:  # df["Interaction"]:
    print(oversamples[inter], inter, rare_inter.loc[inter])
    Xtemp = df[df["Interaction"] == inter]
    aug_rows = pd.DataFrame(np.nan, index=range(oversamples[inter]), columns=df.columns)
    print(f"Xtemp.shape: {Xtemp.shape}")

    for i in range(oversamples[inter]):
        pp = i % Xtemp.shape[0]  # this way, if 1669 < 10000, we restart
        row = np.array(Xtemp.iloc[pp])  # take a row
        aug_rows.iloc[i] = mod_row(
            row, factor=DAUG_factor, idxs=col_to_mod_idx, verbose=False
        )  # mod it

    DAUG = pd.concat([data_aug, aug_rows], ignore_index=True)

### Numerical

In [6]:
df_num = df[
    [
        "s_rsa",
        "s_up",
        "s_down",
        "s_phi",
        "s_psi",
        "s_a1",
        "s_a2",
        "s_a3",
        "s_a4",
        "s_a5",
        "t_rsa",
        "t_up",
        "t_down",
        "t_phi",
        "t_psi",
        "t_a1",
        "t_a2",
        "t_a3",
        "t_a4",
        "t_a5",
        "dist",
        "Interaction",
    ]
]

# Saving the numerical dataset
df_num.to_csv(path_to_data / "df_num.csv", index=False)

df_num.head()

,s_rsa,s_up,s_down,s_phi,s_psi,s_a1,s_a2,s_a3,s_a4,s_a5,...,t_down,t_phi,t_psi,t_a1,t_a2,t_a3,t_a4,t_a5,dist,Interaction
0,0.304,7.0,10.0,-1.107,-0.733,1.357,-1.453,1.477,0.113,-0.837,...,6.0,-1.102,-0.657,1.831,-0.561,0.533,-0.277,1.648,6.292980,HBOND
1,0.086,14.0,24.0,-2.214,2.152,0.931,-0.179,-3.005,-0.503,-1.853,...,16.0,-1.350,2.242,0.336,-0.417,-1.673,-1.474,-0.078,7.907008,HBOND
2,0.000,17.0,17.0,-1.093,-0.837,-1.343,0.465,-0.862,-1.020,-0.255,...,23.0,-1.043,-0.817,1.831,-0.561,0.533,-0.277,1.648,6.614870,HBOND
3,0.742,1.0,13.0,-1.182,-0.456,1.050,0.302,-3.656,-0.259,-3.242,...,16.0,-1.034,-0.718,-0.591,-1.302,-0.733,1.570,-0.146,6.087462,HBOND
4,0.634,0.0,15.0,-2.661,2.866,1.357,-1.453,1.477,0.113,-0.837,...,12.0,-2.073,2.008,-1.337,-0.279,-0.544,1.242,-1.262,5.471278,HBOND


### Categorical

In [7]:
df_cat = df[
    [
        "s_ch",
        "s_ins",
        "s_resn",
        "s_ss3",
        "s_ss8",
        "t_ch",
        "t_ins",
        "t_resi",
        "t_resn",
        "t_ss3",
        "t_ss8",
        "Interaction",
    ]
]

print(f"Before OneHot: {df_cat.shape}")

col_for_OHE = ["s_ss8", "s_ss3", "t_ss8", "t_ss3"]


encoder = ce.OneHotEncoder(cols=col_for_OHE, use_cat_names=True)

df_cat = encoder.fit_transform(df_cat)

print(f"After OneHot: {df_cat.shape}")

# Saving the categorical dataset
df_cat.to_csv(path_to_data / "df_cat.csv", index=False)

df_cat.head()

Before OneHot: (2468259, 12)
After OneHot: (2468259, 32)


,s_ch,s_ins,s_resn,s_ss3_H,s_ss3_L,s_ss3_nan,s_ss8_H,s_ss8_E,s_ss8_B,s_ss8_S,...,t_ss8_H,t_ss8_E,t_ss8_B,t_ss8_T,t_ss8_-,t_ss8_S,t_ss8_G,t_ss8_I,t_ss8_nan,Interaction
0,A,,E,1,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,HBOND
1,A,,Q,1,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,HBOND
2,A,,C,1,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,HBOND
3,A,,D,1,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,HBOND
4,A,,E,1,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,HBOND


In [8]:
df_complete = pd.concat([df_num.drop("Interaction", axis=1), df_cat], axis=1)

# Saving the dataset
df_complete.to_csv(path_to_data / "df_complete.csv", index=False)

print(f"df_complete.shape: {df_complete.shape}")

df_complete.head()

df_complete.shape: (2468259, 53)


,s_rsa,s_up,s_down,s_phi,s_psi,s_a1,s_a2,s_a3,s_a4,s_a5,...,t_ss8_H,t_ss8_E,t_ss8_B,t_ss8_T,t_ss8_-,t_ss8_S,t_ss8_G,t_ss8_I,t_ss8_nan,Interaction
0,0.304,7.0,10.0,-1.107,-0.733,1.357,-1.453,1.477,0.113,-0.837,...,1,0,0,0,0,0,0,0,0,HBOND
1,0.086,14.0,24.0,-2.214,2.152,0.931,-0.179,-3.005,-0.503,-1.853,...,0,1,0,0,0,0,0,0,0,HBOND
2,0.000,17.0,17.0,-1.093,-0.837,-1.343,0.465,-0.862,-1.020,-0.255,...,1,0,0,0,0,0,0,0,0,HBOND
3,0.742,1.0,13.0,-1.182,-0.456,1.050,0.302,-3.656,-0.259,-3.242,...,1,0,0,0,0,0,0,0,0,HBOND
4,0.634,0.0,15.0,-2.661,2.866,1.357,-1.453,1.477,0.113,-0.837,...,0,0,1,0,0,0,0,0,0,HBOND
